# Build With India Hackathon - Local Adapted Version
# This version works without Google Colab


In [2]:
import os, sys
HOME = os.getcwd()
print(f"Working directory: {HOME}")

Working directory: c:\Users\admin\Desktop\Build With India\SUBMISSION\code


In [3]:
# Check if GPU is available (optional)
try:
    import torch
    print(f"GPU available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU device: {torch.cuda.get_device_name(0)}")
except:
    print("PyTorch not installed - will install with ultralytics")

GPU available: False


In [4]:
# Install ultralytics (includes PyTorch)
!pip install ultralytics==8.2.103 -q

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103  Python-3.12.5 torch-2.9.1+cpu CPU (Intel Core(TM) i5-2410M 2.30GHz)
Setup complete  (4 CPUs, 7.9 GB RAM, 104.4/292.4 GB disk)


In [5]:
# Skip Google Drive mounting - clone dataset directly
print("Cloning Falcon dataset...")
!git clone https://github.com/duality-robotics/syntheticDataWorks_multiclass.git

Cloning Falcon dataset...
^C


Cloning into 'syntheticDataWorks_multiclass'...
error: RPC failed; curl 56 Recv failure: Connection was reset
error: 280 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [ ]:
# Navigate to dataset folder
os.chdir('./syntheticDataWorks_multiclass/Output')
print(f"Current directory: {os.getcwd()}")
print(f"Files in directory: {os.listdir('.')}")

In [ ]:
# Verify dataset structure
import os
folders = ['train', 'val', 'test']
for folder in folders:
    if os.path.exists(folder):
        images = len(os.listdir(f'{folder}/images')) if os.path.exists(f'{folder}/images') else 0
        labels = len(os.listdir(f'{folder}/labels')) if os.path.exists(f'{folder}/labels') else 0
        print(f"{folder}: {images} images, {labels} labels")
    else:
        print(f"{folder}: folder not found")

In [ ]:
# Start YOLO training (adapted for local execution)
from ultralytics import YOLO

print("Starting YOLO training...")
print("This will take 20-30 minutes with GPU, 1-2 hours with CPU")

# Load model
model = YOLO('yolov8n.pt')  # Downloads automatically

# Train with reduced settings for local execution
results = model.train(
    data='data.yaml',  # Dataset config file
    epochs=50,         # Reduced from 100 for faster training
    imgsz=640,
    batch=8,           # Smaller batch size for local hardware
    name='hackathon_run',
    save=True,
    plots=True
)

In [ ]:
# Evaluate the trained model
print("Evaluating model performance...")
metrics = model.val()

print(f"\nFinal Results:")
print(f"mAP@0.5: {metrics.box.map50:.3f} ({metrics.box.map50*100:.1f}%)")
print(f"mAP@0.5-0.95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")

In [ ]:
# Display results location
import glob

results_dir = 'runs/detect/hackathon_run'
if os.path.exists(results_dir):
    print(f"\nResults saved in: {results_dir}")
    print("\nGenerated files:")
    
    files_to_check = [
        'weights/best.pt',
        'confusion_matrix.png', 
        'results.csv',
        'train_batch0.jpg',
        'val_batch0_pred.jpg'
    ]
    
    for file in files_to_check:
        full_path = os.path.join(results_dir, file)
        if os.path.exists(full_path):
            print(f"✅ {file}")
        else:
            print(f"❌ {file} (not found)")
else:
    print("Results directory not found")

In [ ]:
# Test model on sample images
if os.path.exists('test/images'):
    test_images = glob.glob('test/images/*.png')[:3]
    
    print(f"\nTesting on {len(test_images)} sample images:")
    for img_path in test_images:
        results = model(img_path)
        print(f"Processed: {os.path.basename(img_path)}")
        # results[0].show()  # Uncomment to display images
else:
    print("No test images found")

## Training Complete!

### Next Steps:
1. Copy `runs/detect/hackathon_run/weights/best.pt` to `SUBMISSION/model/`
2. Copy `runs/detect/hackathon_run/confusion_matrix.png` to `SUBMISSION/results/`
3. Copy `runs/detect/hackathon_run/results.csv` to `SUBMISSION/results/`
4. Update `HACKATHON_REPORT.md` with your actual mAP@0.5 score
5. Convert report to PDF and submit

### Performance Target:
- **Competitive**: mAP@0.5 > 50%
- **Excellent**: mAP@0.5 > 70%